# Notebook for developing and simple testing of metadata update

In [1]:
import pyodbc
from ME_Append import get_me_nbr
from MarkLogicConnection import MarkLogicConnection

In [2]:
import os

# Connections

### AIMS

In [3]:
auth_aims = os.environ.get('auth_aims')
aims_user, aims_pass = (auth_aims.split())
AIMS_conn = pyodbc.connect('DSN=aims_prod; UID={}; PWD={}'.format(aims_user, aims_pass))
AIMS_conn.execute('SET ISOLATION TO DIRTY READ;')

### MarkLogic

In [5]:
auth_marklogic = os.environ.get('auth_marklogic_dev')
ml_user, ml_pass = (auth_marklogic.split())

In [6]:
#url = 'http://appd1454:8000/LATEST'

MarkLogicCon = MarkLogicConnection(username=ml_user, password=ml_pass, server='dev')

# Scan Metadata

In [7]:
metadata_uris = MarkLogicCon.get_file_uris()  # default params: database='PhysicianSanctions', collection='json_data'

results len: 100
results len: 100
results len: 100
results len: 100
results len: 100
results len: 100
results len: 100
results len: 100
results len: 100
results len: 100
results len: 100
results len: 100
results len: 100
results len: 100
results len: 100
results len: 100
results len: 100
results len: 100
results len: 100
results len: 100
results len: 100
results len: 100
results len: 100
results len: 100
results len: 100
results len: 100
results len: 100
results len: 100
results len: 100
results len: 100
results len: 100
results len: 100
results len: 100
results len: 100
results len: 100
results len: 100
results len: 100
results len: 100
results len: 100
results len: 100
results len: 100
results len: 100
results len: 100
results len: 100
results len: 100
results len: 100
results len: 100
results len: 100
results len: 100
results len: 100
results len: 100
results len: 100
results len: 100
results len: 100
results len: 100
results len: 100
results len: 100
results len: 100
results len: 1

### Filtering out SummaryList files

In [8]:
temp = [f for f in metadata_uris if 'summarylist' not in f]
len(temp)

7265

In [9]:
len(metadata_uris)

8471

In [10]:
metadata_uris = temp

In [11]:
from pprint import pprint
import json

In [12]:
def get_state(uri):
    '/json/2019.10.30_OH_NA_SL/schwartz_2172.json'
    
    _sl_index = uri.index('_SL/')
    
    return uri[_sl_index-5:_sl_index-3]

In [14]:
for uri in metadata_uris[:5]:
    print(uri)
    
    json_data = json.loads(MarkLogicCon.get_file(uri=uri))
    
    me_nbr = json_data['sanction']['physician']['me']
    
    # if ME is filled, we don't need to do anything
    if me_nbr in ['None', None, 'Null', 'none', 'null', '']:
        print('ME NUMBER NOT FILLED.')
        pprint(json_data)
        # if license is filled, we can perform a search for ME number
        lic_nbr = json_data['sanction']['physician']['license']
        if lic_nbr in ['None', None, 'Null', 'none', 'null', '']:
            print('LICENSE NUMBER NOT FILLED.')
        else:
            print('SEARCHING FOR ME_NBR')
            
            first = json_data['sanction']['physician']['name_first']
            last = json_data['sanction']['physician']['name_last']
            state = get_state(uri)
            me_nbr = get_me_nbr(first=first, last=last, state=state, lic_nbr=lic_nbr, con=AIMS_conn)
            
            print('ME_NBR:', me_nbr)
            
            if me_nbr is not None:
                print('UPDATING FILE')
                
                MarkLogicCon.update_metadata_me_nbr(uri=uri, me_nbr=me_nbr)
    
    print('#'*80)

/json/2019.10.30_OH_NA_SL/schwartz_2172.json
################################################################################
/json/2019.02.19_IN_NA_SL/wills_2730.json
################################################################################
/json/2020.02.05_IN_NA_SL/shoemaker_3600.json
ME NUMBER NOT FILLED.
{'app': {'assignment': {'me': None, 'pas': None},
         'meta': {'alert': None, 'note': None},
         'status': {'me_complete': None,
                    'pas_complete': None,
                    'state': 'incomplete'}},
 'class': 'sanction',
 'sanction': {'batch': '2020.02.05_IN_NA_SL',
              'date_board_order': None,
              'date_effective': None,
              'decision': None,
              'id': '2020.02.05_IN_NA_SL/shoemaker_3600',
              'method': 'web',
              'orig': 'results/results_02_05_2020_09_00PM/IN_SummaryList_09_14PM_February_05_2020/IN-Michael_Shoemaker-02_05_2020_BO.pdf',
              'pas_code': None,
              'phys

In [55]:
ex = '/pdf/2019.10.30_OH_NA_SL/schwartz_2172.pdf'
f = MarkLogicCon.get_file(ex)

In [56]:
with open('ex_pdf.pdf', 'wb') as file:
    file.write(f)

In [25]:
import os

In [ ]:
# 35050898

In [30]:
import pandas as pd

In [57]:
sql_template = \
"""
SELECT 
    ek.key_type_val,
    nm.first_nm,
    nm.last_nm,
    lic.lic_nbr,
    lic.state_cd

FROM
    person_name_et nm

    INNER JOIN
    entity_key_et ek
    ON nm.entity_id = ek.entity_id

    INNER JOIN
    license_lt lic
    ON lic.entity_id = ek.entity_id

WHERE
    ek.key_type          = "ME"    AND
    UPPER(nm.first_nm)   = "{}"    AND
    UPPER(nm.last_nm)    = "{}"
;
"""
first = 'Paul'
last = 'Schwartz'

sql = sql_template.format(first.upper(), last.upper())

data = pd.read_sql(con=AIMS_conn, sql=sql)

In [58]:
data

,key_type_val,first_nm,last_nm,lic_nbr,state_cd
0,03508601084,Paul,Schwartz,014390,CT
1,03508661273,Paul,Schwartz,G00017813,CA
2,03508661273,Paul,Schwartz,60099056,NY
3,02608820364,Paul,Schwartz,G00060459,CA
4,02608820364,Paul,Schwartz,,KY
5,01102871177,Paul,Schwartz,ME0064947,FL
6,01102871177,Paul,Schwartz,60177537,NY
7,02301741033,Paul,Schwartz,D17118,MD
8,03503020229,Paul,Schwartz,046684,CT
9,03503020229,Paul,Schwartz,25MA07887700,NJ


In [59]:
data[data['state_cd']=='OH']

,key_type_val,first_nm,last_nm,lic_nbr,state_cd
16,03841831585,Paul,Schwartz,35050898,OH


In [11]:
uri = '/json/2019.10.30_OH_NA_SL/schwartz_2172.json'
lic_nbr = '35050898'
MarkLogicCon.set_me_nbr(uri=uri, me_nbr=None)
MarkLogicCon.set_lic_nbr(uri=uri, lic_nbr=lic_nbr)

204
204
